#Assignment -3
#Sharmila Gowri Velugula - ku3936

#**3A) Seq2Seq Language Translation model - Character Level**

Import marathi text file. [mar.txt]

This is basic character-level recurrent sequence-to-sequence model. We apply it to translating short English sentences into short marathi sentences, character-by-character. 

**Summary of the algorithm**

We start with input sequences from a domain (e.g. English sentences) and corresponding target sequences from another domain (e.g. French sentences).
An encoder LSTM turns input sequences to 2 state vectors (we keep the last LSTM state and discard the outputs).
A decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future, a training process called "teacher forcing" in this context. It uses as initial state the state vectors from the encoder. Effectively, the decoder learns to generate targets[t+1...] given targets[...t], conditioned on the input sequence.
In inference mode, when we want to decode unknown input sequences, we: - Encode the input sequence into state vectors - Start with a target sequence of size 1 (just the start-of-sequence character) - Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character - Sample the next character using these predictions (we simply use argmax). - Append the sampled character to the target sequence - Repeat until we generate the end-of-sequence character or we hit the character limit.

#Import Libraries

In [15]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 

#Load Dataset

In [16]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'mar.txt'

#Vectorize the Data

In [17]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
   

In [18]:
len(lines)

40189

In [19]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 19
Max sequence length for outputs: 42


In [20]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])



#Encoder

In [21]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


In [23]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.


In [24]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '$': 5, ',': 6, '-': 7, '.': 8, ':': 9, '?': 10, 'ँ': 11, 'ं': 12, 'ः': 13, 'अ': 14, 'आ': 15, 'इ': 16, 'ई': 17, 'उ': 18, 'ऊ': 19, 'ऋ': 20, 'ए': 21, 'ऐ': 22, 'ऑ': 23, 'ओ': 24, 'औ': 25, 'क': 26, 'ख': 27, 'ग': 28, 'घ': 29, 'च': 30, 'छ': 31, 'ज': 32, 'झ': 33, 'ञ': 34, 'ट': 35, 'ठ': 36, 'ड': 37, 'ढ': 38, 'ण': 39, 'त': 40, 'थ': 41, 'द': 42, 'ध': 43, 'न': 44, 'प': 45, 'फ': 46, 'ब': 47, 'भ': 48, 'म': 49, 'य': 50, 'र': 51, 'ल': 52, 'ळ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, '़': 59, 'ा': 60, 'ि': 61, 'ी': 62, 'ु': 63, 'ू': 64, 'ृ': 65, 'ॅ': 66, 'े': 67, 'ै': 68, 'ॉ': 69, 'ो': 70, 'ौ': 71, '्': 72, '।': 73, '०': 74, '१': 75, '२': 76, '३': 77, '४': 78, '५': 79, '६': 80, '७': 81, '८': 82, '९': 83, '\u200d': 84}


In [25]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]



#Decoder

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Build the Model

In [26]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 85)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 335872      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  350208      input_4[0][0]                    
                                                                 lstm_2[0][1]          

#Fit the Model

In [27]:
from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=3)

In [28]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,callbacks=[early_stopping_monitor],verbose=1,
          validation_split=0.2)
# Save model
#model.save('s2s.h5')

Epoch 1/100
125/125 [==============================] - 38s 304ms/step - loss: 1.5773 - accuracy: 0.6388 - val_loss: 1.5887 - val_accuracy: 0.5970
Epoch 2/100
125/125 [==============================] - 37s 298ms/step - loss: 1.1743 - accuracy: 0.7045 - val_loss: 1.2131 - val_accuracy: 0.6850
Epoch 3/100
125/125 [==============================] - 37s 298ms/step - loss: 0.9341 - accuracy: 0.7552 - val_loss: 1.0683 - val_accuracy: 0.7174
Epoch 4/100
125/125 [==============================] - 37s 298ms/step - loss: 0.8175 - accuracy: 0.7788 - val_loss: 0.9711 - val_accuracy: 0.7338
Epoch 5/100
125/125 [==============================] - 37s 297ms/step - loss: 0.7497 - accuracy: 0.7934 - val_loss: 0.9186 - val_accuracy: 0.7491
Epoch 6/100
125/125 [==============================] - 37s 298ms/step - loss: 0.6937 - accuracy: 0.8079 - val_loss: 0.8668 - val_accuracy: 0.7613
Epoch 7/100
125/125 [==============================] - 37s 299ms/step - loss: 0.6487 - accuracy: 0.8203 - val_loss: 0.8273 -

In [29]:
model.save('s2s_char.h5')

#Inference Model


1. Encode input and retrieve initial decoder state
2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
3.Repeat with the current target token and current states

In [30]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [31]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

#Predictions

In [32]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [33]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: मला बघ.

-
Input sentence: Run!
Decoded sentence: वाचा!

-
Input sentence: Run!
Decoded sentence: वाचा!

-
Input sentence: Run!
Decoded sentence: वाचा!

-
Input sentence: Run!
Decoded sentence: वाचा!

-
Input sentence: Who?
Decoded sentence: कोण आहे?

-
Input sentence: Wow!
Decoded sentence: वाचा!

-
Input sentence: Fire!
Decoded sentence: वाचा!

-
Input sentence: Fire!
Decoded sentence: वाचा!

-
Input sentence: Help!
Decoded sentence: वाचा!

-
Input sentence: Help!
Decoded sentence: वाचा!

-
Input sentence: Jump!
Decoded sentence: उडी राहा!

-
Input sentence: Jump!
Decoded sentence: उडी राहा!

-
Input sentence: Jump.
Decoded sentence: टॉमला वाचव.

-
Input sentence: Jump.
Decoded sentence: टॉमला वाचव.

-
Input sentence: Stop!
Decoded sentence: वाजवा!

-
Input sentence: Stop!
Decoded sentence: वाजवा!

-
Input sentence: Wait!
Decoded sentence: वाचा!

-
Input sentence: Wait!
Decoded sentence: वाचा!

-
Input sentence: Hello!
Decoded sentence: वाचा!

